# Log in API
Use this API call to get an authentication token and start a session. Authentication not required.

#### POST /token/{username}/{password}/({authentication_id})

**URL**: base_url/V1/Session
> **Note:** The base_url is http(s):// "IP address of your NetBrain Web API Server" /ServicesAPI/API

**Parameters**(*required)

|Name|Description|
|------|------|
|username*(string)  | the username to log into your NetBrain domain.  |
|password*(string)  | the password to log into your NetBrain domain.  |
|authentication_id (string)  | the authentication ID of an external user through SSO, LDAP/AD or TACACS. This parameter is only required for an external user and the value is alias. |

**Response**

|Name|Description|
|------|------|
|token(string)  | The returned authentication token.  |
|statusCode(integer)  | The returned status code of executing the API.  |
|statusDescription(string)  | The explanation of the status code. |

**Example**


In [ ]:
# import python modules 
import requests
import time
import urllib3
import pprint
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Set the request parameters
# Eg. User name="NetBrain", Password="Netbrain" for this code sample.
user = "NetBrain"      
pwd = "Netbrain"      
nb_url = "http(s)://<IP address of NetBrain Web Server>/"           

# define a login function 
def loginSession(nb_url, username, password, authentication_id = None):
    full_url = nb_url + "/ServicesAPI/API/V1/Session"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    if not authentication_id:
        body = {
                'username': username,
                'password': password
                }
    else:
        body = {
                'username': username,
                'password': password,
                'authentication_id': authentication_id,
                }
    try:
        # Do the HTTP request
        response = requests.post(full_url, headers=headers, data = json.dumps(body), verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            token = response.json()["token"]
            return (token)
        else:
            return ("Get token failed! - " + str(response.text))
    except Exception as e: return (str(e))
    
token = loginSession(nb_url, user, pwd)
print("Token: "+token)# print out the token you got as a string 

<font color='red'>Question: Response shows totally three parameters should be returned But API founction shows only one return value?</font> 

# Log out API
Use this API to log out from the current session.

#### DELETE /{token}

**URL**: base_url/V1/Session


**Parameters**(*required)

|Name|Description|
|------|------|
|token*(string)  | input the token received from login.  |

**Response**

|Name|Description|
|------|------|
|statusCode(integer)  | The returned status code of executing the API.  |
|statusDescription(string)  | The explanation of the status code. |

**Example**

In [ ]:
# Set the request parameters
token = "301ebdd8-0045-429d-8807-c51b8db7f5f0"
nb_url = "http(s)://<IP address of NetBrain Web Server>/"

# Logout NetBrain and release system token.
def logoutSession(nb_url ,token):
    full_url = nb_url + "/ServicesAPI/API/V1/Session"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    data = {"token":token}
    try:
        # Do the HTTP request
        response = requests.delete(full_url, data=json.dumps(data), headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            return ("Session logout successfully!")
        else:
            return ("Session logout failed! - " + str(response.text))
    except Exception as e: return (str(e))
    
result = logoutSession(nb_url ,token)
print(result)# print out the result of whether log out successfully.

# Get all accessible tenants API
This function returns a list of accessible tenants (including tenant ID and names). The returned tenant list varies by the user privileges you use to log in. To retrieve a full list of all available tenants, you must log in with admin permissions. Authentication required.

#### GET /tenants/{token}

**URL**: base_url/V1/CMDB/Tenants

**Parameters**(*required)

|Name|Description|
|------|------|
|token*(string)  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |

**Response**

|Name|Description|
|------|------|
|tenants(array)  | A list of all accessible tenants.  |
|tenantId(string)  |The tenant ID. |
|tenantName(string)  | The tenant name.  |
|statusCode(integer)  | Code issued by NetBrain server indicating the execution result.  |
|statusDescription(string)  | The explanation of the status code.  |

**Example:**

In [ ]:
# Set the request parameters
token = "301ebdd8-0045-429d-8807-c51b8db7f5f0"
nb_url = "http(s)://<IP address of NetBrain Web Server>/"

# Get all Tenants ID and Name.
def getTenants(nb_url, token):
    full_url = nb_url + "/ServicesAPI/API/V1/CMDB/Tenants"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    try:
        # Do the HTTP request
        response = requests.get(full_url, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return (result["tenants"])
        else:
            return ("Get tenants failed! - " + str(response.text))
    except Exception as e: return (str(e))

allT = getTenants(nb_url, token)
for i in range(len(allT)):
    print(allT[i].tenantName) # print out the name of all accessible tenants 

# Get all accessible domains of a tenant API
This function returns a list of accessible domains in a specific tenant. The returned accessible domains vary by the user privileges you use to log in. To retrieve a full list of domains in a specified tenant, you must log in with system admin or tenant admin permissions.

#### GET /domains/{token}/{tenantId}

**URL**: base_url/V1/CMDB/Domains

**Parameters**(*required)

|Name|Description|
|------|------|
|token*(string)  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
|tenantId(string)  | Unique identifier for the tenant from which you desire to retrieve the domain information. tenantId can be retrieved from get all accessible tenants.  |

**Response**

|Name|Description|
|------|------|
|domains(array)  | A list of all accessible domains.  |
|domainId(string)  |The domain ID. |
|domainName(string)  | The domain name.  |
|statusCode(integer)  | Code issued by NetBrain server indicating the execution result.  |
|statusDescription(string)  | The explanation of the status code.  |

**Example:**

In [ ]:
# Set the request parameters
token = "301ebdd8-0045-429d-8807-c51b8db7f5f0"
nb_url = "http(s)://<IP address of NetBrain Web Server>/"
tenantId = "3e75247a-309c-4231-96a5-823b6cb1e78d"

# Get all domains info. If tenantId is None, will present all domains.
def getDomains(nb_url, token, tenantId=None):
    full_url = nb_url + "/ServicesAPI/API/V1/CMDB/Domains"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    data = {"tenantId": tenantId}
    try:
        # Do the HTTP request
        response = requests.get(full_url, params=data, headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            # Decode the JSON response into a dictionary and use the data
            result = response.json()
            return (result["domains"])
        else:
            return ("Get domains failed! - " + str(response.text))
    except Exception as e: return (str(e))
    
allD = getDomains(nb_url, token)
for i in range(len(allD)):
    print(allD[i].domainName) # print out the name of all accessible tenants 

# Get all accessible domains of a tenant API
Use this API to specify a domain to work on to get or set NetBrain data by associating domainID to the current session.

#### PUT /domains/{token}/{tenantId}/{domainId}

**URL**: base_url/V1/CMDB/Domains

**Parameters**(*required)

|Name|Description|
|------|------|
|token*(string)  | The token can be obtained by sending a POST request to the log in session endpoint and provide valid credentials.  |
|tenantId*(string)  | Unique identifier for the tenant from which you desire to retrieve the domain information. tenantId can be retrieved from get all accessible tenants.  |
|domainId*(string)  | input the ID of the target domain. Get a domain ID by using the API [Get all accessible domains of a tenant.](https://www.netbraintech.com/docs/ie71/help/get-all-accessible-domains-of-tenant.htm) |

**Response**

|Name|Description|
|------|------|
|statusCode(integer)  | Code issued by NetBrain server indicating the execution result.  |
|statusDescription(string)  | The explanation of the status code.  |

**Example:**

In [ ]:
# Set the request parameters
token = "301ebdd8-0045-429d-8807-c51b8db7f5f0"
nb_url = "http(s)://<IP address of NetBrain Web Server>/"
tenantId = "3e75247a-309c-4231-96a5-823b6cb1e78d"
domainId = "4e75247a-309c-4231-96a5-823b6cb1e78d"

# Login selected domain
def loginDomain(nb_url, token, tenantId, domainId):
    full_url = nb_url + "/ServicesAPI/API/V1/Session/CurrentDomain"
    # Set proper headers
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    headers["Token"]=token
    body = {
            "tenantId": tenantId,
            "domainId": domainId
        }
    try:
        # Do the HTTP request
        response = requests.put(full_url, data=json.dumps(body), headers=headers, verify=False)
        # Check for HTTP codes other than 200
        if response.status_code == 200:
            return ("loginDomain: Login successfully!")
        elif response.status_code != 200:
            return ("Login failed! - " + str(response.text))
    except Exception as e: return (str(e))

result = loginDomain(nb_url, token, tenantId, domainId)
print(result)# print out the login result of one particular domain. 